# Functions

In [1]:
import time
from typing import Any, Dict

import numpy as np
import torch
import torch.nn as nn
from torch.autograd.profiler import profile as profile_ag
from torch.profiler import profile, record_function, ProfilerActivity


MB = 1024 * 1024
ms = 1000


def profile_net(model: nn.Module, *model_input) -> Dict[str, Any]:
    """A tool that allows you to collect data on the performance of a neural network.
    Args:
        model: PyTorch model to profile.
        model_input: Input to the model (data, parameters, etc.)
    Returns:
        Information about shape, time, memory and model size.
    """
    with profile(
            activities=[ProfilerActivity.CUDA], 
            record_shapes=True
        ) as prof:
        with record_function("model_inference"):
            model(*model_input)
    
    print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))
    
    torch.cuda.synchronize()
    with profile_ag(use_cuda=True, profile_memory=True) as prof:
        start_time = time.perf_counter()
        _, _, _, _, output = model(*model_input)
        end_time = time.perf_counter()

    return {
        'output_shape': output.shape,
        'elapsed_time, ms': np.round((end_time - start_time), 4),
        'cpu_memory, Mb': np.round(prof.total_average().cpu_memory_usage, 2),
        'gpu_memory, Mb': np.round(prof.total_average().cuda_memory_usage, 2),
        'model_size, Mb': np.round(
            sum(p.numel() * p.element_size() for p in model.parameters()) / MB, 2
        ),
    }
    


/home/dvkorshunov/miniconda/envs/bts/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cpu



# BTS

In [9]:
import sys
bts_dir = "/home/dvkorshunov/mono_depth/monocular_depth_estimation/networks/bts"
bts_torch_dir = bts_dir + "/pytorch"
bts_pretrains_dir = bts_dir + "/models"
checkpoint_path = bts_pretrains_dir + "bts_eigen_v2_pytorch_densenet121/model"
dataset_dir = "/home/dvkorshunov/mono_depth/monocular_depth_estimation/datasets/datasets/kitti/"
sys.path.insert(1, bts_torch_dir)

In [10]:
from collections import namedtuple

arguments = (
    "encoder "
    "data_path "
    "dataset "
    "filenames_file "
    "model_name "
    "checkpoint_path "
    "input_height "
    "input_width "
    "max_depth "
    "bts_size "
    "do_kb_crop"
)
Params = namedtuple('Params', arguments)
params = Params(
    encoder="densenet121_bts",
    data_path=dataset_dir,
    dataset="kitti",
    filenames_file= bts_dir + "/train_test_inputs/eigen_test_files_with_gt_copy.txt",
    model_name="bts_eigen_v2_pytorch_densenet121",
    checkpoint_path=checkpoint_path,
    input_height=352,
    input_width=1216,
    max_depth=80,
    bts_size=512,
    do_kb_crop=True
)

# Load pretrains

In [18]:
! mkdir -p {bts_pretrains_dir}
! wget -nc https://cogaplex-bts.s3.ap-northeast-2.amazonaws.com/bts_eigen_v2_pytorch_densenet121.zip -O {bts_pretrains_dir + "/bts_eigen_v2_pytorch_densenet121.zip"}
! unzip -o {bts_pretrains_dir + "/bts_eigen_v2_pytorch_densenet121.zip"} -d {bts_pretrains_dir}    

File ‘/home/dvkorshunov/mono_depth/monocular_depth_estimation/networks/bts/models/bts_eigen_v2_pytorch_densenet121.zip’ already there; not retrieving.
Archive:  /home/dvkorshunov/mono_depth/monocular_depth_estimation/networks/bts/models/bts_eigen_v2_pytorch_densenet121.zip
  inflating: /home/dvkorshunov/mono_depth/monocular_depth_estimation/networks/bts/models/bts_eigen_v2_pytorch_densenet121/arguments_train_eigen.txt  
  inflating: /home/dvkorshunov/mono_depth/monocular_depth_estimation/networks/bts/models/bts_eigen_v2_pytorch_densenet121/bts_eigen_v2_pytorch_densenet121.py  
  inflating: /home/dvkorshunov/mono_depth/monocular_depth_estimation/networks/bts/models/bts_eigen_v2_pytorch_densenet121/model  


# Profile

In [25]:
from tqdm.notebook import tqdm
from torch.autograd import Variable

from bts_dataloader import BtsDataLoader
from bts import BtsModel

def get_num_lines(file_path):
    f = open(file_path, 'r')
    lines = f.readlines()
    f.close()
    return len(lines)

dataloader = BtsDataLoader(params, mode="test")
    
model = BtsModel(params=params)
model = torch.nn.DataParallel(model)

checkpoint = torch.load(params.checkpoint_path)
model.load_state_dict(checkpoint['model'])
model.eval()
model.cuda()

FileNotFoundError: [Errno 2] No such file or directory: '/home/dvkorshunov/mono_depth/monocular_depth_estimation/networks/bts/modelsbts_eigen_v2_pytorch_densenet121/model'

In [ ]:
num_params = sum([np.prod(p.size()) for p in model.parameters()])
print("Total number of parameters: {}".format(num_params))

num_test_samples = get_num_lines(params.filenames_file)

with open(params.filenames_file) as f:
    lines = f.readlines()

print('now testing {} files with {}'.format(num_test_samples, params.checkpoint_path))

pred_depths = []
pred_8x8s = []
pred_4x4s = []
pred_2x2s = []
pred_1x1s = []

start_time = time.time()
profile_data = []
with torch.no_grad():
    for num, sample in enumerate(tqdm(dataloader.data)):
        image = Variable(sample['image'].cuda())
        focal = Variable(sample['focal'].cuda())
        
        # warm up
        if num == 30:
            print(profile_net(model, image, focal))
            break
        # Predict
        lpg8x8, lpg4x4, lpg2x2, reduc1x1, depth_est = model(image, focal)
        pred_depths.append(depth_est.cpu().numpy().squeeze())
        pred_8x8s.append(lpg8x8[0].cpu().numpy().squeeze())
        pred_4x4s.append(lpg4x4[0].cpu().numpy().squeeze())
        pred_2x2s.append(lpg2x2[0].cpu().numpy().squeeze())
        pred_1x1s.append(reduc1x1[0].cpu().numpy().squeeze())

elapsed_time = time.time() - start_time
print('Elapesed time: %s' % str(elapsed_time))
print('Done.')

In [ ]:
# import pandas as pd

# df = pd.DataFrame(profile_data)
# df["elapsed_time, ms"]